# Установка OpenAI библиотеки

In [ ]:
!pip install git+https://github.com/openai/whisper.git 
!pip install tiktoken
!pip install setuptools-rust

# Установка библиотек для скачивания и обработки видео с Rutube

In [ ]:
!pip install rutube-downloader m3u8 alive_progress audio-separator scipy
!pip uninstall torch -y
!pip install torch==2.0.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip uninstall onnxruntime -y
!pip install onnxruntime-gpu

In [ ]:
import torch
import onnxruntime

print('Cuda is available:', torch.cuda.is_available())
print('Count cuda devices:', torch.cuda.device_count())
print('ONNXRuntime works on', onnxruntime.get_device())

In [ ]:
from rutube import Rutube
import os, logging

rt = Rutube('https://rutube.ru/video/925830c78b4a4f865206b3645ee983a3/')
fname = str(rt.playlist[-1])
if not os.path.exists(fname+'.mp4'):
    print('Downloading...')
    rt.playlist[-1].download()
print('Video downloaded')
    
import subprocess
import numpy as np
#os.remove(f'{fname}.wav')
if not os.path.exists(fname+'.wav'):
    print('Getting audio from mp4...')
    subprocess.call(['ffmpeg', '-i', f'{fname}.mp4', '-vn', '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '2', f'{fname}.wav'])
print('Got audio from mp4')
    
from audio_separator import Separator

print('Separating audio...')
# Initialize the Separator with the audio file and model name
separator = Separator(f'{fname}.wav', model_name='UVR_MDXNET_KARA_2', use_cuda=True, log_level=logging.FATAL, model_file_dir='asm/', primary_stem_path='background.wav', secondary_stem_path='vocal.wav')

# Perform the separation
primary_stem_path, secondary_stem_path = separator.separate()

print(f'Primary stem saved at {primary_stem_path}')
print(f'Secondary stem saved at {secondary_stem_path}')

print('Audio separated')

print('Converting to mono...')
from pydub import AudioSegment
sound = AudioSegment.from_wav(secondary_stem_path)
sound = sound.set_channels(1)
sound.export(secondary_stem_path, format="wav")
print('Converted to mono')

In [ ]:
import whisper, json

model = whisper.load_model("medium", download_root="openai")
result = model.transcribe(secondary_stem_path, task='translate', language="Russian")
print(result["text"], json.dumps(result))

In [ ]:
!pip install speakerpy
!sudo apt-get install libportaudio2

In [ ]:
from speakerpy.lib_speak import Speaker

speaker = Speaker(model_id="v3_en", language="en", speaker="en_0", device="cuda")
speaker.to_mp3(text=result['text'], name_text=fname, sample_rate=48000, audio_dir='out_mp3', speed=1.0)

In [ ]:
import os

audio_file = 'out_mp3/'+[i for i in os.listdir('out_mp3') if i != 'cache' and i.endswith('.mp3')][0]
print(audio_file)

In [ ]:
from pydub import AudioSegment
sound1 = AudioSegment.from_file(primary_stem_path)
sound2 = AudioSegment.from_file(audio_file)
sound2 = sound2 - 10
sound1 = sound1 - 15
combined = sound1.overlay(sound2)
combined.export(fname+'.new.mp3', format='mp3')

In [ ]:
#!pip install moviepy

In [ ]:
# from moviepy.editor import *
# videoclip = VideoFileClip(fname+'.mp4')
# audioclip = AudioFileClip(fname+".new.mp3")
# # new_audioclip = CompositeAudioClip([audioclip])
# videoclip = videoclip.set_audio(audioclip)
# videoclip.write_videofile(fname+".translated.mp4", fps=60.0)
# # videoclip.ipython_display(fps=60) 

import subprocess

# Input video file
video = fname+'.mp4'

# Input audio file
audio = fname+'.new.mp3'

# Output video file
output = fname+'.translated.mp4'

# Replace audio in output video
subprocess.call(['ffmpeg', '-i', video, '-i', audio, '-c:v', 'copy', '-map', '0:v:0', '-map', '1:a:0', '-shortest', '-y', output])

In [ ]:
!zip -r ./* -9